In [45]:
#import needed modules, tree-sitter and tree-sitter-java have already been pip installed in environment
from typing import Generator
import tree_sitter_java
from tree_sitter import Language, Parser, Tree, Node
import torch
from unixcoder import UniXcoder

In [46]:
#create parser capable of understanding java language
parser = Parser(Language(tree_sitter_java.language()))

In [47]:
tree = parser.parse(bytes('int a = 1', "utf8"))
#create sample tree and build traverse_tree function capable of walking through tree-sitter structure representing java
def traverse_tree(tree: Tree) -> Generator[Node, None, None]:
    cursor = tree.walk()

    visited_children = False
    while True:
        if not visited_children:
            yield cursor.node
            if not cursor.goto_first_child():
                visited_children = True
        elif cursor.goto_next_sibling():
            visited_children = False
        elif not cursor.goto_parent():
            break


node_names = map(lambda node: node.type, traverse_tree(tree))

#show traverse tree works as expected
assert list(node_names) == [
    'program',
    'local_variable_declaration',
    'integral_type',
    'int',
    'variable_declarator',
    'identifier',
    '=',
    'decimal_integer_literal',
    ';'
]

In [48]:
#download data and print out
with open('AntBuildTrigger.java') as ivy:
    data = ivy.read()
print(data)

/*
 *  Licensed to the Apache Software Foundation (ASF) under one or more
 *  contributor license agreements.  See the NOTICE file distributed with
 *  this work for additional information regarding copyright ownership.
 *  The ASF licenses this file to You under the Apache License, Version 2.0
 *  (the "License"); you may not use this file except in compliance with
 *  the License.  You may obtain a copy of the License at
 *
 *      https://www.apache.org/licenses/LICENSE-2.0
 *
 *  Unless required by applicable law or agreed to in writing, software
 *  distributed under the License is distributed on an "AS IS" BASIS,
 *  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 *  See the License for the specific language governing permissions and
 *  limitations under the License.
 *
 */
package org.apache.ivy.ant;

import java.io.File;
import java.util.ArrayList;
import java.util.Collection;
import java.util.Map;

import org.apache.ivy.core.IvyContext;
import org.ap

In [49]:
#test parser correctly works on larger java file
tree = parser.parse(bytes(data, "utf8"))
node_names = map(lambda node: node.type, traverse_tree(tree))
unique_nodes = []
for i in node_names:
    unique_nodes.append(i)
test_out = set(unique_nodes)
print(test_out)
#will be looking at package_declaration, "class_declaration, and method_declaration identifiers for analzying granularity at 
#package, class, and method levels respectively

#for taken level will go through all tokens, just printed out unique here for test, excluding block_comment and line_comment types
#which represent comments in the java code

{'+', 'string_literal', '!', '&&', 'generic_type', 'import', 'throw_statement', 'catch', 'boolean_type', 'cast_expression', 'enhanced_for_statement', 'try_statement', 'modifiers', 'this', 'try', 'class_declaration', 'void_type', 'super_interfaces', 'false', ':', 'formal_parameter', 'true', 'if', '!=', 'scoped_identifier', 'ternary_expression', 'class_body', 'program', 'public', '"', 'object_creation_expression', 'new', 'field_declaration', 'extends', '<', '==', 'assignment_expression', 'throw', 'if_statement', 'return_statement', 'formal_parameters', 'string_fragment', 'argument_list', ';', 'private', 'catch_formal_parameter', 'method_declaration', 'identifier', '>', 'superclass', 'parenthesized_expression', ')', 'catch_type', 'else', '+=', 'binary_expression', '(', 'scoped_type_identifier', '=', 'type_identifier', 'package_declaration', '{', 'expression_statement', 'type_arguments', 'block_comment', 'for', 'null_literal', 'field_access', 'local_variable_declaration', '?', 'package', '

In [50]:
#count and print number of packages in tree
#initial output should be one as tree equal to single package 'AntBuildTrigger.java' for testing
i = 0
for node in traverse_tree(tree):
    if node.type == 'package_declaration':
        print('i =', i)
        print(data[node.start_byte:node.end_byte])
        i += 1
i

i = 0
package org.apache.ivy.ant;


1

In [51]:
#loads if have machine with Nvidia.cude installed will use, otherwise will use cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#initializes unixcoder
model = UniXcoder("microsoft/unixcoder-base")
#sets model to device chosen above
model.to(device)

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [52]:
#copied from Cynthia's github
#code for collecting data from single Java file

results = {} # Store data here
id = 0 # Each embedding will have its own ID

# Run through every node in the parse tree
parser = Parser(Language(tree_sitter_java.language()))
tree = parser.parse(bytes(data, 'utf8'))
for node in traverse_tree(tree):        
    # Store results
    if node.type == 'class_declaration':
        level = 'class'
    elif node.type == 'method_declaration':
        level = 'method'
    elif (len(node.children) == 0) and (node.type != 'block_comment' and node.type != 'line_comment'): # Leaf node that is not a comment
        level = 'token'
    else: # If node does not fit one of the above levels, move on to the next
        continue
    
    # Get code embedding
    indices = (node.start_byte, node.end_byte) # Start and end indices to parse code fragments out of the source code
    code = data[indices[0] : indices[1]]
    with torch.no_grad():
        torch.cuda.empty_cache() #empty cache of gradients to imrpove efficiency
        tokens_ids = model.tokenize([code]) #convert code strings into list of token id's model can use as input
        source_ids = torch.tensor(tokens_ids).to(device) #moves input to where model running
        tokens_embeddings, code_embedding = model(source_ids) #get embeddings
        torch.cuda.empty_cache() #clear cache

    results[id] = (code_embedding.tolist(), level, indices) # Add code embedding and related information
    id += 1 # ID for next embedding


In [53]:
#write results into file named 'results.json', should appear in directory
import json
with open("results.json", 'w') as out:
    json.dump(results, out)

In [54]:
#import modules for downloading data from github
from github import Github
from github import Auth
import requests

In [55]:
#take fine grain token and access Github
token = '' #put in own token
auth = Auth.Token(token)
g = Github(auth=auth)

In [56]:
#download contents from apache/ant-ivy repository, show repository
repo = g.get_repo(full_name_or_id='apache/ant-ivy')
repo

Repository(full_name="apache/ant-ivy")

In [57]:
#gets contents out of repository
contents = repo.get_contents('src/java/org/apache/ivy')[0]
contents

ContentFile(path="src/java/org/apache/ivy/Ivy.java")

In [58]:
#show path to contents in repository
contents.path

'src/java/org/apache/ivy/Ivy.java'

In [59]:
response = requests.get(contents.download_url)
print(response.text)

/*
 *  Licensed to the Apache Software Foundation (ASF) under one or more
 *  contributor license agreements.  See the NOTICE file distributed with
 *  this work for additional information regarding copyright ownership.
 *  The ASF licenses this file to You under the Apache License, Version 2.0
 *  (the "License"); you may not use this file except in compliance with
 *  the License.  You may obtain a copy of the License at
 *
 *      https://www.apache.org/licenses/LICENSE-2.0
 *
 *  Unless required by applicable law or agreed to in writing, software
 *  distributed under the License is distributed on an "AS IS" BASIS,
 *  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 *  See the License for the specific language governing permissions and
 *  limitations under the License.
 *
 */
package org.apache.ivy;

import java.io.File;
import java.io.IOException;
import java.io.InputStream;
import java.net.URL;
import java.text.ParseException;
import java.text.SimpleDat

In [60]:
# Get the download urls of every java file in a subdirectory
user = 'apache'
repo = 'ant-ivy'
sub_dir = 'src/java/org/apache/ivy'
repo = g.get_repo(full_name_or_id = user + '/' + repo) #gets apache/ant-ivy repository in this case

def traverse_github(urls:list, dir:str): #goes through everything in directory
    for content in repo.get_contents(dir):
        if len(content.name.split('.')) > 1: # Base case: Content is file
            # Only return if filename has .java extension
            if content.name.split('.')[-1] == 'java': #only addes java file
                urls.append(content.download_url)
        else: # Recursive case: Content is folder
            traverse_github(urls, content.path)

urls = [] #define empty urls list
traverse_github(urls, sub_dir) #apply to fill list with all relevant urls from apache/ant-ivy repository

In [62]:
#show content of urls is correct
print(urls)

['https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy14.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Main.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AddPathTask.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntBuildTrigger.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntCallTrigger.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntMessageLogger.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntWorkspaceResolver.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/BuildOBRTask.java', 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apac

In [63]:
#created indexed dictionaries with urls
indexed_urls = {idx:url for idx,url in enumerate(urls)}
indexed_urls

{0: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy.java',
 1: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy14.java',
 2: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Main.java',
 3: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AddPathTask.java',
 4: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntBuildTrigger.java',
 5: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntCallTrigger.java',
 6: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntMessageLogger.java',
 7: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntWorkspaceResolver.java',
 8: 'https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/BuildOBRTask.java',
 9: 'https://raw.githubusercontent.com/

In [64]:
#number of urls
len(indexed_urls)

488

In [65]:
#dump urls into json document
import json
json.dump(indexed_urls, open("download_urls.json", 'w'))

In [66]:
import requests
import re

In [67]:
#read data from file
import json
files = json.load(open("download_urls.json"))

In [68]:
#create dictionary with package as key and id's of apache/ant-ivy files where they are
packages = {}
for id, file in files.items():
    response = requests.get(file)
    match = re.search(r"package [^\s]*;", response.text)
    pkg = match.group()
    if pkg not in packages.keys():
        packages[pkg] = [id]
    else:
        packages[pkg].append(id)

In [69]:
#print out packages and indexes of files in which they're downloaded
for item in packages.items():
    print(item)

('package org.apache.ivy;', ['0', '1', '2'])
('package org.apache.ivy.ant;', ['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52'])
('package org.apache.ivy.core;', ['53', '54', '55', '56', '57', '58', '59'])
('package org.apache.ivy.core.cache;', ['60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73'])
('package org.apache.ivy.core.check;', ['74', '75'])
('package org.apache.ivy.core.deliver;', ['76', '77', '78', '79', '80'])
('package org.apache.ivy.core.event;', ['81', '82', '83', '84', '85'])
('package org.apache.ivy.core.event.download;', ['86', '87', '88', '89', '90'])
('package org.apache.ivy.core.event.publish;', ['91', '92', '93'])
('package org.apache.ivy.core.event.resolve;', ['94', '95', '96', '97', '98', '99'

In [70]:
#create strings to build directories to store json files
copy = files
directory_list = []
for id, file in copy.items():
    file_adjust = file.replace('https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/','')
    file_adjust_again = adjusted[:-5] #remove.java for name of directory
    directory_list.append(file_adjust_again)
print(directory_list)  

['ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHandlerRegistry', 'ivy/util/url/URLHa

In [71]:
#turn directory strings into dictionary for easy access
directory_indexed_urls = {idx:url for idx,url in enumerate(listy)}
directory_indexed_urls

{0: 'ivy/Ivy',
 1: 'ivy/Ivy14',
 2: 'ivy/Main',
 3: 'ivy/ant/AddPathTask',
 4: 'ivy/ant/AntBuildTrigger',
 5: 'ivy/ant/AntCallTrigger',
 6: 'ivy/ant/AntMessageLogger',
 7: 'ivy/ant/AntWorkspaceResolver',
 8: 'ivy/ant/BuildOBRTask',
 9: 'ivy/ant/ConvertManifestTask',
 10: 'ivy/ant/EmptyFileSet',
 11: 'ivy/ant/FixDepsTask',
 12: 'ivy/ant/IvyAntSettings',
 13: 'ivy/ant/IvyAntVariableContainer',
 14: 'ivy/ant/IvyArtifactProperty',
 15: 'ivy/ant/IvyArtifactReport',
 16: 'ivy/ant/IvyBuildList',
 17: 'ivy/ant/IvyBuildNumber',
 18: 'ivy/ant/IvyCacheFileset',
 19: 'ivy/ant/IvyCachePath',
 20: 'ivy/ant/IvyCacheTask',
 21: 'ivy/ant/IvyCheck',
 22: 'ivy/ant/IvyCleanCache',
 23: 'ivy/ant/IvyConfigure',
 24: 'ivy/ant/IvyConflict',
 25: 'ivy/ant/IvyConvertPom',
 26: 'ivy/ant/IvyDeliver',
 27: 'ivy/ant/IvyDependency',
 28: 'ivy/ant/IvyDependencyArtifact',
 29: 'ivy/ant/IvyDependencyConf',
 30: 'ivy/ant/IvyDependencyExclude',
 31: 'ivy/ant/IvyDependencyInclude',
 32: 'ivy/ant/IvyDependencyTree',
 33: '

In [72]:
from traceback import print_exc #help understand errors with exception handling

In [73]:
import os #needed to help make directories

In [74]:
#taking code for embeddings from single java file and making into function
#modeled after generate_code_embeddings from Cynthia's code
def get_embeddings(source_code:str, model:UniXcoder, device:str = None):
    results = {} # Store data here
    id = 0 # Each embedding will have its own ID

# Run through every node in the parse tree
    parser = Parser(Language(tree_sitter_java.language()))
    tree = parser.parse(bytes(data, 'utf8'))
    for node in traverse_tree(tree):        
        # Store results
        if node.type == 'class_declaration':
            level = 'class'
        elif node.type == 'method_declaration':
            level = 'method'
        elif (len(node.children) == 0) and (node.type != 'block_comment' and node.type != 'line_comment'): # Leaf node that is not a comment
            level = 'token'
        else: # If node does not fit one of the above levels, move on to the next
            continue
    
        # Get code embedding
        indices = (node.start_byte, node.end_byte) # Start and end indices to parse code fragments out of the source code
        code = data[indices[0] : indices[1]]
        with torch.no_grad():
            torch.cuda.empty_cache() #empty cache of gradients to imrpove efficiency
            tokens_ids = model.tokenize([code]) #convert code strings into list of token id's model can use as input
            source_ids = torch.tensor(tokens_ids).to(device) #moves input to where model running
            tokens_embeddings, code_embedding = model(source_ids) #get embeddings
            torch.cuda.empty_cache() #clear cache

        results[id] = (code_embedding.tolist(), level, indices) # Add code embedding and related information
        id += 1 # ID for next embedding
    return results
    

In [75]:
#modeled off embed_all_files from CodeEmeddingsGenerator.py on Cynthia's Github, some adjustments made for new
#directory system instructed to install
def embed_all_files(files:dict) -> None:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UniXcoder("microsoft/unixcoder-base")
    model.to(device)
    # Run through every file
    for id, url in files.items():
        class_embed = {}
        method_embed = {}
        token_embed = {}
        print(f'Generating code embeddings with #{id}: {url}')
        try:
            response = requests.get(url) # Get file
            results = get_embeddings(response.text, model, device)
            place = test_indexed_urls[int(id)]
            if not os.path.exists(place): #make directory based on where file located in ant-ivy if doesn't already exist
                os.makedirs(place)
            for i in results.keys(): #create seperate list of embeddings for each type
                work = results[i]
                if work[1] == 'class':
                    class_embed[i] = work
                elif work[1] == 'method':
                    method_embed[i] =  work
                elif work[1] == 'token':
                    token_embed[i] = work                   
            #add each type into its own file
            json.dump(class_embed, open(f'{place}/class_embed_{id}.json', 'w')) # Save as json
            json.dump(method_embed, open(f'{place}/method_embed_{id}.json', 'w')) # Save as json
            json.dump(token_embed, open(f'{place}/token_embed_{id}.json', 'w')) # Save as json
        except Exception:
            print(f'Something has gone wrong with file {id}: {url}')
            print_exc()
    return

In [76]:
#create directories filled with json files
java_files = json.load(open("download_urls.json"))
embed_all_files(java_files)

Generating code embeddings with #0: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy.java
Generating code embeddings with #1: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Ivy14.java
Generating code embeddings with #2: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/Main.java
Generating code embeddings with #3: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AddPathTask.java
Generating code embeddings with #4: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntBuildTrigger.java
Generating code embeddings with #5: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntCallTrigger.java
Generating code embeddings with #6: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/ant/AntMessageLogger.java
Generating code embeddings with #7: https://raw.githubusercont

Generating code embeddings with #59: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/RelativeUrlResolver.java
Generating code embeddings with #60: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/cache/ArtifactOrigin.java
Generating code embeddings with #61: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/cache/CacheDownloadOptions.java
Generating code embeddings with #62: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/cache/CacheMetadataOptions.java
Generating code embeddings with #63: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/cache/CacheResourceOptions.java
Generating code embeddings with #64: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/cache/CacheUtil.java
Generating code embeddings with #65: https://raw.githubusercontent.com/apache/ant-ivy/master/src/jav

Generating code embeddings with #112: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/Configuration.java
Generating code embeddings with #113: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/ConfigurationAware.java
Generating code embeddings with #114: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/ConfigurationGroup.java
Generating code embeddings with #115: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/ConfigurationIntersection.java
Generating code embeddings with #116: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/DefaultArtifact.java
Generating code embeddings with #117: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/module/descriptor/DefaultDependencyArtifactDescriptor

Generating code embeddings with #164: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/IvyNode.java
Generating code embeddings with #165: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/IvyNodeBlacklist.java
Generating code embeddings with #166: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/IvyNodeCallers.java
Generating code embeddings with #167: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/IvyNodeEviction.java
Generating code embeddings with #168: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/IvyNodeUsage.java
Generating code embeddings with #169: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/core/resolve/ResolveData.java
Generating code embeddings with #170: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/

Generating code embeddings with #218: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/core/OsgiLatestStrategy.java
Generating code embeddings with #219: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/filter/AndFilter.java
Generating code embeddings with #220: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/filter/CompareFilter.java
Generating code embeddings with #221: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/filter/MultiOperatorFilter.java
Generating code embeddings with #222: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/filter/NotFilter.java
Generating code embeddings with #223: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/filter/OSGiFilter.java
Generating code embeddings with #224: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/a

Generating code embeddings with #272: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/util/ParseUtil.java
Generating code embeddings with #273: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/util/Version.java
Generating code embeddings with #274: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/util/VersionComparator.java
Generating code embeddings with #275: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/util/VersionRange.java
Generating code embeddings with #276: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/osgi/util/ZipUtil.java
Generating code embeddings with #277: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/IvySettingsAware.java
Generating code embeddings with #278: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/circ

Generating code embeddings with #324: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/namespace/NameSpaceHelper.java
Generating code embeddings with #325: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/namespace/Namespace.java
Generating code embeddings with #326: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/namespace/NamespaceRule.java
Generating code embeddings with #327: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/namespace/NamespaceTransformer.java
Generating code embeddings with #328: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/parser/AbstractModuleDescriptorParser.java
Generating code embeddings with #329: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/parser/ModuleDescriptorParser.java
Generating code embeddings with #330: https:/

Generating code embeddings with #376: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/repository/vfs/VfsRepository.java
Generating code embeddings with #377: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/repository/vfs/VfsResource.java
Generating code embeddings with #378: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/repository/vsftp/VsftpRepository.java
Generating code embeddings with #379: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/repository/vsftp/VsftpResource.java
Generating code embeddings with #380: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/resolver/AbstractPatternsBasedResolver.java
Generating code embeddings with #381: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/resolver/AbstractResolver.java
Generating code embeddings wi

Generating code embeddings with #428: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/version/VersionMatcher.java
Generating code embeddings with #429: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/plugins/version/VersionRangeMatcher.java
Generating code embeddings with #430: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/tools/analyser/DependencyAnalyser.java
Generating code embeddings with #431: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/tools/analyser/JarJarDependencyAnalyser.java
Generating code embeddings with #432: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/tools/analyser/JarModule.java
Generating code embeddings with #433: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/tools/analyser/JarModuleFinder.java
Generating code embeddings with #434: https://raw.githubuserconte

Generating code embeddings with #484: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/util/url/TimeoutConstrainedURLHandler.java
Generating code embeddings with #485: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/util/url/URLHandler.java
Generating code embeddings with #486: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/util/url/URLHandlerDispatcher.java
Generating code embeddings with #487: https://raw.githubusercontent.com/apache/ant-ivy/master/src/java/org/apache/ivy/util/url/URLHandlerRegistry.java
